In [9]:
import pandas as pd
import numpy as np
import os, datetime, time, pyodbc, math, itertools
from mysql import *
from sqlalchemy import create_engine
import df_to_sql.df_to_sql as sq
import create_table.tbl_mysql as myq
import create_table.tbl_mssql as msq
import osq as oq


def drop_columns(df,ls):
    cols = df.columns.to_list()
    for i in cols:
        for n in ls:
            if i == n:
                df = df.drop([i], axis=1)
    else:
        return df

def rmv_space_in_cols(df):
    for i in df:
        if i.find(' ') != 0:
            ni = i.replace(' ', '_')
            df = df.rename(columns={i:ni})
    else:
        return df
            
def mod_datetime(df):
    df = df.convert_dtypes()
    cols = df.columns.to_list()
    for i in cols:
        if df[i].dtypes == 'string':
            try:
                df[i] = df.apply(lambda x : pd.to_datetime(x[i]), axis = 1)
                try:
                    df[i] = df.apply(lambda x : pd.to_datetime(x[i]).strftime("%Y-%m-%d %H:%M:%S"), axis = 1)
                except:
                    try:
                        df[i] = df.apply(lambda x : pd.to_datetime(x[i]).strftime("%Y-%m-%d"), axis = 1)
                    except:
                        pass
            except:
                pass
    else:
        try:
            df = df.convert_dtypes()
            return df
        except:
            return df
        
def modstr(strval):
    if isinstance(strval, str):
        try:
            s1 = strval.replace("'","\\'")
            return s1
        except:
            return strval
    else:
        return strval
    
def inupd(df,tbl,update_by=False):
    df = df.fillna('$')
    ls = []
    if update_by == False or update_by == []:
        cols = df.columns.to_list()
        vals = df.values.tolist()
        sq = 'insert into ' + tbl + " "
        cnt = 1
        for i in range(len(vals)):
            L2ST = ','.join(["'" + str(n) + "'" for indx, n in enumerate(vals[i]) if (n!='$')])
            x2 = sq + '(' + ','.join(cols) + ') values ('+ L2ST + ')'
            ls.append(x2)
            cnt = cnt + 1
        else:
            return ls
    else:
        for item in itertools.chain(cols, vals):
            print(item)
                   
def insert_into(df):
    df = df.fillna('$')
    ls = []
    cols = df.columns.to_list()
    vals = df.values.tolist()
    for i,v in enumerate(vals):
        cL = ''
        vL = ''
        for i2,v2 in enumerate(v):
            if cL == '' and v2 != '$':
                cL = str(cols[i2])
                vL = "'" + str(modstr(v2)) + "'"
            elif cL != '' and v2 != '$':
                cL = cL + "," + cols[i2]
                vL = vL + "," + "'" + str(modstr(v2)) + "'"
        else:
            tmp = " (" + cL + ") VALUES (" + vL + ")"
            ls.append(tmp)
    else:
        return ls
        
def update_into(df,bycols=[],seperator=',', operator = '='):
    ls = []
    cols = df.columns.to_list()
    vals = df.values.tolist()
    cnt = 0
    for i,v in enumerate(vals):
        cL = ''
        for i2,v2 in enumerate(v):
            st = str(cols[i2]) + ' ' + operator + " '" + str(modstr(v2)) + "'"
            if cL == '' and v2 != '$' and cols[i2] not in bycols:
                cL = st
            elif cL != '' and v2 != '$' and cols[i2] not in bycols:
                cL = cL + seperator + st
        else:
            ls.append(cL)
    else:
        return ls

def is_table_exist(conn, tbl):
    sql = 'select 1 from ' + tbl
    try:
        df = pd.read_sql(sql, con = self.conn)
        print('table name : ', tbl, ' (exist)')
        return 1
    except:
        print('table name :', tbl ,' (does not exist)')
        return 0

def mssql_table_colinfo(table, conn):
    dic = {}
    try:
        qry = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '" + table + "' ORDER BY ORDINAL_POSITION"
        dfx = pd.read_sql(qry, con = conn)
        dbcols = dfx['COLUMN_NAME'].to_list()
        dbcolType = dfx['DATA_TYPE'].to_list()
        dc= zip(dbcols, dbcolType)
        dic = dict(dc)
        return dic
    except:
        return dic

def mysql_table_colinfo(db, table, conn):
    dic = {}
    try:
        qry = 'EXPLAIN ' + db + '.' + table
        dfx = pd.read_sql(qry, con = conn)
        dbcols = dfx['Field'].to_list()
        dbcolType = dfx['Type'].to_list()
        dc= zip(dbcols, dbcolType)
        dic = dict(dc)
        return dic
    except:
        return dic
    
def export_ex(mdf,qdf,tbl,conn):
    df = rmv_space_in_cols(mdf)
    cr = conn.cursor()        
    z = msq.CreateTable_MSSQL(df, tbl, conn)
    cnt = 0
    print(qdf)
    for i in range(len(qdf)):
        cnt = cnt + 1
        if qdf.shape[1] == 2:
            one = qdf.loc[i,'INS']
            two = qdf.loc[i,'UPD']
            try:
                cr.execute(two)
                print('succ : ', i, ' qry ', two)
            except:
                print('failed : ', i, ' qry ', two)
                try:
                    cr.execute(one)
                    print('succ : ', i, ' qry ', one)
                except:
                    print('failed : ', i, ' qry ', one)
        elif qdf.shape[1] == 1:
            one = qdf.loc[i,'INS']
            try:
                cr.execute(one)
                print('succ : ', i, ' qry ', one)
            except:
                print('failed : ', i, ' qry ', one)
    else:
        conn.commit()
        return '1'

class OMSQ1:
    def __init__(self,user='',password='',host='',dbname='',table = False):
        self.U = user
        self.P = password
        self.H = host
        self.D = dbname
        self.tableinfo = {}
        self.conn = ''
        self.cn = ''
        self.dbtype = ''
        self.tbl = table
        self.df = pd.DataFrame([])
        self.qdf = pd.DataFrame([])
        self.qrydfmain = pd.DataFrame([])
        self.qrydf = pd.DataFrame([])
    def oracle_default(self):
        self.conn = cx_Oracle.connect('SOC_READ','soc_read', 'ossam-cluster-scan.robi.com.bd:1721/RBPB.robi.com.bd')
        print(self.conn)
    def oracle_custom(self):
        self.conn = cx_Oracle.connect(self.U,self.P,'ossam-cluster-scan.robi.com.bd:1721/RBPB.robi.com.bd')
        print(self.conn)
    def mssql(self):
        cstr2 = "mssql+pyodbc://" + self.U + ':' + self.P + '@' + self.H + "/" + self.D + "?driver=SQL Server"
        self.conn = create_engine(cstr2)
        self.dbtype = 'MSSQL'
        print('conn succ mssql')
    def pyodbc_mssql(self):
        cstr = "Driver={SQL Server};SERVER=" + self.H + ";DATABASE=" + self.D + ";UID=" + self.U + ";PWD=" + self.P
        self.cn = pyodbc.connect(cstr)
        self.dbtype = 'MSSQL'
        print('conn succ mssql')
    def mysql(self):
        constr = 'mysql+mysqlconnector://' + self.U + ':' + self.P + '@' + self.H + '/' + self.D
        self.conn = create_engine(constr, echo=False)
        self.cn = self.conn.raw_connection()
        self.dbtype = 'MYSQL'
        print('conn succ mysql')
    def table_info(self, tbl=False):
        if tbl == False:
            tbl = self.tbl
        if self.dbtype == 'MSSQL':
            self.tableinfo = mssql_table_colinfo(tbl,self.conn)
        elif self.dbtype == 'MYSQL':
            self.tableinfo = mysql_table_colinfo(self.D, tbl, self.conn)
        else:
            print('No DB initiated yet')
    def ex(self, qry):
        self.qdf = pd.read_sql(qry, con = self.conn)
        return qdf
    def get_conn_oracle(self):
        return self.conn
    def get_conn_other(self):
        return self.cn
    def set_df(self,ndf):
        self.df = ndf
    def check_table(self, tbl):
        return is_table_exist(self.conn,self.table)
    def get_tableinfo(self):
        return self.tableinfo
    def get_qrydf(self):
        return self.qrydf
    def create_tbl(self,df,tbl):
        if self.dbtype == 'MSSQL':
            self.pyodbc_mssql()
            msq.CreateTable_MSSQL(df, tbl, self.cn)
        elif self.dbtype == 'MYSQL':
            myq.CreateTable_MYSQL(connection = self.cn, tablename = tbl, df = df)
    def export_prep(self, ndf, tbl, bycol=[], sep=','):
        self.tbl = tbl
        self.table_info(self.tbl)
        df = ndf.fillna('$')
        df = mod_datetime(df)
        self.qrydfmain = df
        df = rmv_space_in_cols(self.qrydfmain)
        ls1 = insert_into(df)
        lss1 = ['INSERT INTO ' + tbl + i for i in ls1]
        if bycol is None or bycol==[]:
            df1 = pd.DataFrame(ls1, columns=['INS'])
            df1['INS'] = df1.apply(lambda x : 'INSERT INTO ' + tbl + x['INS'], axis = 1)
            self.qrydf = df1
        else:
            ls2 = update_into(df,bycols=bycol)
            dfn = df[bycol]
            ls3 = update_into(dfn,bycols=[],seperator=sep,operator='=')
            lss2 = ['UPDATE ' + tbl + ' SET ' + i + ' WHERE ' + j for i, j in zip(ls2, ls3)]
            self.qrydf = pd.DataFrame(zip(lss2,lss1),columns=['UPD','INS'])
    def export_execute(self, ifexist='fail'):
        try:
            self.qrydfmain.to_sql(self.tbl, con=self.conn, if_exists=ifexist, chunksize=10000)
            print('try success')
        except:
            if self.dbtype == 'MSSQL':
                self.pyodbc_mssql()
            cr = self.cn.cursor()
            for i in range(len(self.qrydf)):
                for j in self.qrydf:
                    q = self.qrydf.loc[i,j]
                    x = q.find(" WHERE ")
                    if x != -1:
                        tb = q.find(self.tbl)
                        qq = 'select 1 from ' + self.tbl + ' ' + q[x:len(q)]
                        ddf = pd.read_sql(qq,con=self.conn)
                        if ddf.shape[0]>0:
                            try:
                                cr.execute(q)
                                break
                            except:
                                print('failed: ', q)
                    else:
                        try:
                            cr.execute(q)
                            break
                        except:
                            print('failed: ', q)
            else:
                self.cn.commit()
                cr.close()
                print('over')
    def cursor_execute(self, qry):
        cr = self.cn.cursor()
        rs = cr.execute(qry)
        return rs
    def engin_execute(self, qry):
        df = pd.read_sql(qry, con= self.conn)
        return df
    
        


#ob1.mssql()
pt = os.getcwd() + '\\T9.csv'
csv1 = "D:\\omEngin\\AOmPy_o_\\csv_o_\\sclick3.csv"
ndf = pd.read_csv(csv1)
col = ndf.columns.to_list()
val = ndf.values.tolist()
ob1 = OMSQ1(user='root',password='admin',host='127.0.0.1:3306',dbname='odb')
#ob1 = OmSq(user='sa',password='1q2w3eaz$',host='192.168.0.102',dbname='omdb')
ob1.mysql()
ob1.create_tbl(ndf,'tbl13')
ob1.export_prep(ndf,'tbl13',['Serial'])
ob1.export_execute()

#update_single(col,val[2],tbl='XX',bycol='Serial',cond_opr=' LIKE ')
#update_single(col,val[2])
#ob1 = OmSq(user='root',password='admin',host='127.0.0.1:3306',dbname='odb')
#ob1 = OmSq(user='sa',password='1q2w3eaz$',host='192.168.0.102',dbname='omdb')
#ob1.mssql()
#cn =ob1.get_conn()
#msq.CreateTable_MSSQL(ndf, 'tbl7', cn)
#x = ob1.cursor_execute('select * from tbl4')
#y = ob1.engin_execute('select * from tbl4')
#ob1.export_prep(ndf,'tbl4')
#print(ob1.get_tableinfo())
#ob1.export_execute()
#ob1.chktbl('tbl2')
#ob1.mssql()
#ob1.insert_into(df,'tbl2')

Exception during reset or similar
Traceback (most recent call last):
  File "e:\python\lib\site-packages\sqlalchemy\pool\base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "e:\python\lib\site-packages\sqlalchemy\pool\base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "e:\python\lib\site-packages\sqlalchemy\dialects\mysql\base.py", line 2475, in do_rollback
    dbapi_connection.rollback()
  File "e:\python\lib\site-packages\mysql\connector\connection_cext.py", line 386, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: MySQL server has gone away


conn succ mysql
CREATE TABLE IF NOT EXISTS `tbl13` ( `Serial` INT NULL DEFAULT NULL, `Severity` TEXT NULL DEFAULT NULL, `Node` TEXT NULL DEFAULT NULL, `Resource` TEXT NULL DEFAULT NULL, `Incident_Owner` TEXT NULL DEFAULT NULL, `FirstOccurrence` DATETIME NULL DEFAULT NULL, `LastOccurrence` DATETIME NULL DEFAULT NULL, `Tally` INT NULL DEFAULT NULL, `CustomAttr11` TEXT NULL DEFAULT NULL, `Summary` TEXT NULL DEFAULT NULL, `AssociatedCR` TEXT NULL DEFAULT NULL, `CustomAttr15` TEXT NULL DEFAULT NULL, `BCCH` TEXT NULL DEFAULT NULL, `AlertKey` TEXT NULL DEFAULT NULL, `TTRequestTime` DATETIME NULL DEFAULT NULL) ENGINE = InnoDB CHARSET=utf32 COLLATE utf32_general_ci
table creation successfull  table name  tbl13
over


In [ ]:
def insert_single(cols,vals,tbl=False):
    cL = ''
    vL = ''
    for i2,v2 in enumerate(vals):
        COL = str(cols[i2])
        if len(str(v2))>0:
            VAL = "'" + str(modstr(v2)) + "'"
            if cL == '':
                cL = COL
                vL = VAL
            else:
                cL = cL + "," + COL
                vL = vL + "," + VAL
    else:
        if tbl == False:
            tmp = " (" + cL + ") VALUES (" + vL + ")"
            return tmp
        else:
            tmp = "INSERT INTO " + tbl + " (" + cL + ") VALUES (" + vL + ")"
            return tmp
        
def update_single(cols, vals, tbl='',bycol='',cond_opr=' LIKE ', sep=','):
    cL = ''
    opr = '='
    cond = ''
    for i2,v2 in enumerate(vals):
        COL = str(cols[i2])
        if bycol == False or bycol=='':
            if len(str(v2))>0:
                VAL = "'" + str(modstr(v2)) + "'"
                if cL == '':
                    cL = COL + ' ' + opr + ' ' + VAL
                else:
                    cL = cL + sep + COL + ' ' + opr + ' ' + VAL
        elif bycol.lower() == COL.lower():
            cond = ' WHERE ' + COL + cond_opr + "'" + str(modstr(v2)) + "'"
        elif bycol.lower() != COL.lower():
            VAL = "'" + str(modstr(v2)) + "'"
            if cL == '':
                cL = COL + ' ' + opr + ' ' + VAL
            else:
                cL = cL + sep + COL + ' ' + opr + ' ' + VAL
    else:
        if len(cond)>1:
            TB = tbl
            if tbl=='':
                TB = '$TABLE$'
            tmp = 'UPDATE ' + TB + ' SET ' + cL + cond
            print(tmp)
        else:
            print(cL)